In [1]:
import pandas as pd
from scipy.stats import chisquare

In [2]:
def get_years_dfs(misinfotext_path: str, fakespeak_path: str):
    misinfotext_file = pd.ExcelFile(misinfotext_path)
    
    for sheet in misinfotext_file.sheet_names:
        df = pd.read_excel(misinfotext_file, sheet_name=sheet)
        df["year"] = sheet
        yield df
    
    fakespeak_file = pd.ExcelFile(fakespeak_path)
    
    for sheet in fakespeak_file.sheet_names:
        df = pd.read_excel(fakespeak_file, sheet_name=sheet)
        df["year"] = sheet
        yield df

In [3]:
def build_chisquare_table(df: pd.DataFrame, entities: list[str]):
    # Aggregate duplicate Entity labels (sum their counts) before reindexing
    counts = df.groupby("Entity", sort=False)["Count"].sum().reindex(entities, fill_value=0)

    table = pd.DataFrame({
        "Entity": entities,
        "Count": counts.values
    })

    table["year"] = df["year"].iloc[0]

    return table


In [4]:
def run_stats_test(misinfotext_path: str, fakespeak_path: str, min_significant_pvalue=0.05):
    years_dfs = list(get_years_dfs(misinfotext_path, fakespeak_path))

    for df in years_dfs:
        df.sort_values("Entity", inplace=True)

    all_entities = sorted(set([
        str(entity) 
        for df in years_dfs
        for entity in df["Entity"] 
    ]))

    tables = [build_chisquare_table(df, all_entities) for df in years_dfs]

    chisquare_result = chisquare([df["Count"] for df in tables])

    num_insignificant_entities = sum(
        1 for pvalue in chisquare_result.pvalue 
        if pvalue > min_significant_pvalue
    )
    num_significant_entities = len(all_entities) - num_insignificant_entities

    print("There are", num_insignificant_entities, "entities that did not change significantly, and", 
          num_significant_entities, "entities that changed significantly")
    print("Total", len(all_entities), "entities")

    return pd.DataFrame({
        "Entity": all_entities,
        "statistic": chisquare_result.statistic,
        "pvalue": chisquare_result.pvalue
    })

#### Body text

In [5]:
results_all_types = run_stats_test(    
    misinfotext_path="./data/MisInfoText/Analysis_output/MisInfoText_named_entities_frequency.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/Fakespeak_named_entities_frequency.xlsx"
)
results_all_types

There are 4373 entities that did not change significantly, and 1927 entities that changed significantly
Total 6300 entities


,Entity,statistic,pvalue
0,.com,45.0,7.657266e-05
1,04,15.0,4.514172e-01
2,"1,000,000",15.0,4.514172e-01
3,1000,15.0,4.514172e-01
4,125th Street,15.0,4.514172e-01
...,...,...,...
6295,İlhan,15.0,4.514172e-01
6296,Α,48.0,2.543044e-05
6297,Ξ,15.0,4.514172e-01
6298,Ο,84.0,1.286887e-11


In [6]:
results_news_and_blog = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/news_and_blog/MisInfoText_news_and_blog_named_entities_frequency.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/news_and_blog/Fakespeak_news_and_blog_named_entities_frequency.xlsx"
)
results_news_and_blog

There are 10280 entities that did not change significantly, and 4863 entities that changed significantly
Total 15143 entities


c:\Users\Adam\AppData\Local\Python\pythoncore-3.11-64\Lib\site-packages\scipy\stats\_stats_py.py:7400: RuntimeWarning: invalid value encountered in divide
  terms = (f_obs - f_exp)**2 / f_exp


,Entity,statistic,pvalue
0,(−)-nicotine,15.000000,4.514172e-01
1,04,15.000000,4.514172e-01
2,1,191.354286,1.205795e-32
3,"1,000,000",79.400000,8.990378e-11
4,"1,000,000,000",46.666667,4.162963e-05
...,...,...,...
15138,Δ,45.000000,7.657266e-05
15139,Ξ,15.000000,4.514172e-01
15140,Ο,84.000000,1.286887e-11
15141,Ρ,30.000000,1.192150e-02


In [7]:
results_press_release = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/press_release/MisInfoText_press_release_named_entities_frequency.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/press_release/Fakespeak_press_release_named_entities_frequency.xlsx"
)
results_press_release

There are 2729 entities that did not change significantly, and 1068 entities that changed significantly
Total 3797 entities


c:\Users\Adam\AppData\Local\Python\pythoncore-3.11-64\Lib\site-packages\scipy\stats\_stats_py.py:7400: RuntimeWarning: invalid value encountered in divide
  terms = (f_obs - f_exp)**2 / f_exp


,Entity,statistic,pvalue
0,.com,105.000000,1.460740e-15
1,1,25.333333,4.564201e-02
2,"1,000,000",21.720000,1.153588e-01
3,"1,000,000,000",17.400000,2.955199e-01
4,10,12.000000,6.790291e-01
...,...,...,...
3792,yield,45.733333,5.861732e-05
3793,young adult,15.333333,4.276838e-01
3794,youth,36.600000,1.447266e-03
3795,½,15.333333,4.276838e-01


In [8]:
results_social_media = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/social_media/MisInfoText_social_media_named_entities_frequency.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/social_media/Fakespeak_social_media_named_entities_frequency.xlsx"
)
results_social_media

There are 73 entities that did not change significantly, and 0 entities that changed significantly
Total 73 entities


,Entity,statistic,pvalue
0,1000,1.0,0.317311
1,2012 VP113,1.0,0.317311
2,American Capitalism,1.0,0.317311
3,Americans,1.0,0.317311
4,Chinese,1.0,0.317311
...,...,...,...
68,scandal,1.0,0.317311
69,speculation,1.0,0.317311
70,trace,1.0,0.317311
71,truth,1.0,0.317311


#### Headline text

In [9]:
results_all_types_headlines = run_stats_test(    
    misinfotext_path="./data/MisInfoText/Analysis_output/MisInfoText_named_entities_frequency_headlines.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/Fakespeak_named_entities_frequency_headlines.xlsx"
)
results_all_types_headlines

There are 817 entities that did not change significantly, and 62 entities that changed significantly
Total 879 entities


,Entity,statistic,pvalue
0,2012 VP113,80.800000,4.985809e-11
1,28th Infantry Division,23.666667,7.097461e-02
2,AAA,15.000000,4.514172e-01
3,AARP,30.000000,1.192150e-02
4,ABC News,15.000000,4.514172e-01
...,...,...,...
874,workforce,15.000000,4.514172e-01
875,world,14.000000,5.255291e-01
876,year,15.000000,4.514172e-01
877,Α,13.000000,6.022979e-01


In [10]:
results_news_and_blog_headlines = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/news_and_blog/MisInfoText_news_and_blog_named_entities_frequency_headlines.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/news_and_blog/Fakespeak_news_and_blog_named_entities_frequency_headlines.xlsx"
)
results_news_and_blog_headlines

There are 1894 entities that did not change significantly, and 165 entities that changed significantly
Total 2059 entities


,Entity,statistic,pvalue
0,...Hits,15.000000,4.514172e-01
1,1,15.000000,4.514172e-01
2,"1,000,000",14.000000,5.255291e-01
3,1000,15.000000,4.514172e-01
4,2012 VP113,98.666667,2.333374e-14
...,...,...,...
2054,wrap,15.000000,4.514172e-01
2055,year,19.545455,1.900714e-01
2056,Élite,15.000000,4.514172e-01
2057,Α,13.571429,5.582493e-01


In [11]:
results_press_release_headlines = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/press_release/MisInfoText_press_release_named_entities_frequency_headlines.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/press_release/Fakespeak_press_release_named_entities_frequency_headlines.xlsx"
)
results_press_release_headlines

There are 414 entities that did not change significantly, and 24 entities that changed significantly
Total 438 entities


,Entity,statistic,pvalue
0,"1,000,000",15.000000,0.451417
1,2012 VP113,30.000000,0.011921
2,28th Infantry Division,23.666667,0.070975
3,AARP,30.000000,0.011921
4,Adult,15.000000,0.451417
...,...,...,...
433,waste,15.000000,0.451417
434,web page,15.000000,0.451417
435,work,15.000000,0.451417
436,world,15.000000,0.451417


In [12]:
results_social_media_headlines = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/social_media/MisInfoText_social_media_named_entities_frequency_headlines.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/social_media/Fakespeak_social_media_named_entities_frequency_headlines.xlsx"
)
results_social_media_headlines

There are 4 entities that did not change significantly, and 0 entities that changed significantly
Total 4 entities


,Entity,statistic,pvalue
0,Evidence,1.0,0.317311
1,Lunar Roving Vehicle,1.0,0.317311
2,Moon Landing,1.0,0.317311
3,sodium,3.0,0.083265
